In [1]:
with open('news', 'r') as f:
    text = f.read()
    news = text.split("\n\n")
    count = {'sport': 0, 'world': 0, "us": 0, "business": 0, "health": 0, "entertainment": 0, "sci_tech": 0}
    for news_item in news:
        lines = news_item.split("\n")
        print(lines[6])
        file_to_write = open('data/' + lines[6] + '/' + str(count[lines[6]]) + '.txt', 'w+')
        count[lines[6]] = count[lines[6]] + 1
        file_to_write.write(news_item)  # python will convert \n to os.linesep
        file_to_write.close()


sport
business
sport
entertainment
us
entertainment
world
business
business
world
us
sport
entertainment
sport
world
health
world
world
us
entertainment
sport
us
us
sport
entertainment
world
business
sport
world
sci_tech
sport
entertainment
business
us
sport
world
world
sport
us
world
sci_tech
sport
health
entertainment
business
us
entertainment
entertainment
sport
sport
entertainment
us
entertainment
business
us
sport
sport
sport
entertainment
business
business
sport
entertainment
us
world
sport
sci_tech
sport
us
sport
business
sport
us
business
us
us
us
world
entertainment
business
world
business
sport
sport
sport
business
world
health
sport
sci_tech
world
entertainment
entertainment
sport
world
world
health
health
business
world
sport
sport
sport
sport
health
world
sport
sport
world
sport
world
world
health
world
world
sport
business
us
us
world
sport
health
business
sport
sport
world
world
us
sport
us
world
business
entertainment
us
entertainment
sci_tech
us
us
business
us
entertai

business
world
sci_tech
business
entertainment
us
us
business
world
business
entertainment
sport
business
sci_tech
sport
entertainment
entertainment
business
health
business
sport
health
health
sport
sci_tech
health
sci_tech
sport
us
us
business
world
health
sport
sport
us
sport
sport
entertainment
health
sport
entertainment
world
sci_tech
us
us
us
world
business
sci_tech
sport
sport
business
sport
sport
business
world
entertainment
world
sport
health
entertainment
world
us
sci_tech
sport
world
world
world
world
sport
entertainment
health
entertainment
business
sci_tech
sport
us
sport
us
entertainment
health
sport
sport
world
entertainment
business
us
world
world
business
world
world
entertainment
world
sci_tech
world
sci_tech
business
world
business
sport
business
health
entertainment
sport
sport
entertainment
sport
us
us
us
entertainment
world
sport
sport
world
business
sci_tech


IndexError: list index out of range

In [2]:
import pandas
import glob

category_list = ["sport", "world", "us", "business", "health", "entertainment", "sci_tech"]
directory_list = ["data/sport/*.txt", "data/world/*.txt","data/us/*.txt","data/business/*.txt","data/health/*.txt","data/entertainment/*.txt","data/sci_tech/*.txt",]

text_files = list(map(lambda x: glob.glob(x), directory_list))
text_files = [item for sublist in text_files for item in sublist]

training_data = []


for t in text_files:
    f = open(t, 'r')
    f = f.read()
    t = f.split('\n')
    training_data.append({'data' : t[0] + ' ' + t[1], 'flag' : category_list.index(t[6])})
    
training_data[0]

{'data': "court agrees to expedite n.f.l.'s appeal the decision means a ruling could be made nearly two months before the regular season begins, time for the sides to work out a deal without delaying the season.",
 'flag': 0}

In [3]:
training_data = pandas.DataFrame(training_data, columns=['data', 'flag'])
training_data.to_csv("train_data.csv", sep=',', encoding='utf-8')
print(training_data.data.shape)


(1433,)


In [4]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer


#GET VECTOR COUNT
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(training_data.data)

#SAVE WORD VECTOR
pickle.dump(count_vect.vocabulary_, open("count_vector.pkl","wb"))

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer

#TRANSFORM WORD VECTOR TO TF IDF
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#SAVE TF-IDF
pickle.dump(tfidf_transformer, open("tfidf.pkl","wb"))

In [6]:
# Multinomial Naive Bayes

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

#clf = MultinomialNB().fit(X_train_tfidf, training_data.flag)
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.flag, test_size=0.25, random_state=42)
clf = MultinomialNB().fit(X_train, y_train)

#SAVE MODEL
pickle.dump(clf, open("nb_model.pkl", "wb"))

In [9]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

category_list = ["sport", "world", "us", "business", "health", "entertainment", "sci_tech"]

docs_new = "Messi joins other football team"
docs_new = [docs_new]

#LOAD MODEL
loaded_vec = CountVectorizer(vocabulary=pickle.load(open("count_vector.pkl", "rb")))
loaded_tfidf = pickle.load(open("tfidf.pkl","rb"))
loaded_model = pickle.load(open("nb_model.pkl","rb"))

X_new_counts = loaded_vec.transform(docs_new)
X_new_tfidf = loaded_tfidf.transform(X_new_counts)
predicted = loaded_model.predict(X_new_tfidf)

print(category_list[predicted[0]])






sport


In [10]:
predicted = loaded_model.predict(X_test)
result_bayes = pandas.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})
result_bayes.to_csv('res_bayes.csv', sep = ',')

for predicted_item, result in zip(predicted, y_test):
    print(category_list[predicted_item], ' - ', category_list[result])



world  -  world
sport  -  world
business  -  business
world  -  world
business  -  business
sport  -  sport
sport  -  sport
sport  -  sport
sport  -  us
sport  -  sci_tech
business  -  business
sport  -  entertainment
business  -  business
sport  -  sport
world  -  world
world  -  world
sport  -  entertainment
sport  -  sport
world  -  world
business  -  us
world  -  world
world  -  world
sport  -  health
sport  -  sport
business  -  sci_tech
sport  -  sport
sport  -  entertainment
business  -  business
business  -  business
business  -  business
world  -  world
sport  -  sport
sport  -  entertainment
sport  -  business
sport  -  entertainment
world  -  world
world  -  sport
sport  -  sport
sport  -  us
sport  -  sport
sport  -  us
sport  -  entertainment
sport  -  us
world  -  world
business  -  business
business  -  business
sport  -  us
business  -  business
sport  -  health
sport  -  us
world  -  world
sport  -  sport
business  -  sci_tech
business  -  business
business  -  busines

In [12]:
from sklearn.metrics import confusion_matrix  

confusion_mat = confusion_matrix(y_test,predicted)
print(confusion_mat)

[[86  1  0  0  0  0  0]
 [ 6 66  0  1  0  0  0]
 [29  7  3  2  0  0  0]
 [17  3  0 46  0  0  0]
 [18  2  1  2  0  0  0]
 [32  1  0  3  0  4  0]
 [17  4  0  8  0  0  0]]


In [13]:
from sklearn.neural_network import MLPClassifier

clf_neural = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(15,), random_state=1)

X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.flag, test_size=0.25, random_state=42)

clf_neural.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(15,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [14]:
pickle.dump(clf_neural, open("softmax.pkl", "wb"))

In [15]:
predicted = clf_neural.predict(X_test)
result_softmax = pandas.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})
result_softmax.to_csv('res_softmax.csv', sep = ',')

for predicted_item, result in zip(predicted, y_test):
    print(category_list[predicted_item], ' - ', category_list[result])


world  -  world
health  -  world
business  -  business
world  -  world
business  -  business
sport  -  sport
health  -  sport
sport  -  sport
us  -  us
health  -  sci_tech
business  -  business
health  -  entertainment
business  -  business
sport  -  sport
world  -  world
us  -  world
health  -  entertainment
health  -  sport
business  -  world
us  -  us
world  -  world
world  -  world
sci_tech  -  health
sport  -  sport
health  -  sci_tech
sport  -  sport
health  -  entertainment
business  -  business
us  -  business
business  -  business
world  -  world
sport  -  sport
entertainment  -  entertainment
health  -  business
sci_tech  -  entertainment
world  -  world
world  -  sport
sport  -  sport
sport  -  us
sport  -  sport
health  -  us
entertainment  -  entertainment
health  -  us
world  -  world
business  -  business
health  -  business
us  -  us
business  -  business
health  -  health
health  -  us
world  -  world
sport  -  sport
business  -  sci_tech
business  -  business
business

In [16]:
from sklearn import svm
clf_svm = svm.LinearSVC()
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, training_data.flag, test_size=0.25, random_state=42)
clf_svm.fit(X_train_tfidf, training_data.flag)
pickle.dump(clf_svm, open("svm.pkl", "wb"))

In [17]:
predicted = clf_svm.predict(X_test)
result_svm = pandas.DataFrame( {'true_labels': y_test,'predicted_labels': predicted})
result_svm.to_csv('res_svm.csv', sep = ',')
for predicted_item, result in zip(predicted, y_test):
    print(category_list[predicted_item], ' - ', category_list[result])

world  -  world
world  -  world
business  -  business
world  -  world
business  -  business
sport  -  sport
sport  -  sport
sport  -  sport
us  -  us
sci_tech  -  sci_tech
business  -  business
entertainment  -  entertainment
business  -  business
sport  -  sport
world  -  world
world  -  world
entertainment  -  entertainment
sport  -  sport
world  -  world
us  -  us
world  -  world
world  -  world
health  -  health
sport  -  sport
sci_tech  -  sci_tech
sport  -  sport
entertainment  -  entertainment
business  -  business
business  -  business
business  -  business
world  -  world
sport  -  sport
entertainment  -  entertainment
business  -  business
entertainment  -  entertainment
world  -  world
sport  -  sport
sport  -  sport
us  -  us
sport  -  sport
us  -  us
entertainment  -  entertainment
us  -  us
world  -  world
business  -  business
business  -  business
us  -  us
business  -  business
health  -  health
us  -  us
world  -  world
sport  -  sport
sci_tech  -  sci_tech
business  